## Import test data

In [137]:
import yfinance as yf
import pandas as pd

df = yf.download("AAPL", start="2013-12-15", end="2023-02-01", interval='1d')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-12-16,19.822144,20.094286,19.821787,19.910713,17.526363,282592800
2013-12-17,19.850357,19.980000,19.763571,19.821072,17.447458,229902400
2013-12-18,19.632143,19.694643,19.242857,19.670357,17.314783,565863200
2013-12-19,19.625000,19.642857,19.418928,19.445000,17.116423,320308800
2013-12-20,19.479643,19.700357,19.457857,19.607857,17.259777,436413600
...,...,...,...,...,...,...
2023-01-25,140.889999,142.429993,138.809998,141.860001,141.643738,65799300
2023-01-26,143.169998,144.250000,141.899994,143.960007,143.740540,54105100
2023-01-27,143.160004,147.229996,143.080002,145.929993,145.707520,70492800


## Signal Function

In [138]:
def signal_generator(df):
    
    Open = df.Open.iloc[-1]
    Close = df.Close.iloc[-1]
    prev_Open = df.Open.iloc[-2]
    prev_Close = df.Close.iloc[-2]
    
    # Bearish Engulfing
    if (Open > Close and prev_Open < prev_Close):
        if (Close < prev_Open and Open >= prev_Close):
            return 1
        else:
            return 0

    # Bullish Engulfing 
    elif (Open < Close and prev_Open > prev_Close):
        if (Close > prev_Open and Open <= prev_Close):
            return 2 
        else: return 0
    
    # No clear pattern
    else:
        return 0

signal = [0]
for i in range(1,len(df)):
    df_signal = df[i-1:i+1]
    signal.append(signal_generator(df_signal))
df["signal"] = signal

In [139]:
df.signal.value_counts()

0    2125
1      89
2      83
Name: signal, dtype: int64

In [140]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,signal
Date,,,,,,,
2013-12-16,19.822144,20.094286,19.821787,19.910713,17.526363,282592800,0
2013-12-17,19.850357,19.980000,19.763571,19.821072,17.447458,229902400,0
2013-12-18,19.632143,19.694643,19.242857,19.670357,17.314783,565863200,0
2013-12-19,19.625000,19.642857,19.418928,19.445000,17.116423,320308800,0
2013-12-20,19.479643,19.700357,19.457857,19.607857,17.259777,436413600,0


## Backtest Strategy

In [134]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import SMA

class MyStrategy(Strategy):  
    
    def init(self):
        super().init()
        self.signal = self.I(lambda: self.data.signal)

    def next(self):
        super().next()
        if self.signal==2:
            self.buy()
        elif self.signal==1:
            self.sell()


class MySMAStrategy(Strategy):
    
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
        
    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


## Engulfing Strategy

In [135]:
backtest = Backtest(df, MyStrategy, cash=10_000, commission = 0.002, exclusive_orders=True)
stats = backtest.run()
stats

Start                     2013-12-16 00:00:00
End                       2023-01-31 00:00:00
Duration                   3333 days 00:00:00
Exposure Time [%]                   99.521114
Equity Final [$]                  3977.989786
Equity Peak [$]                  11744.095246
Return [%]                         -60.220102
Buy & Hold Return [%]              624.685208
Return (Ann.) [%]                   -9.618452
Volatility (Ann.) [%]               26.222249
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -67.856605
Avg. Drawdown [%]                  -10.209047
Max. Drawdown Duration     3143 days 00:00:00
Avg. Drawdown Duration      368 days 00:00:00
# Trades                                  172
Win Rate [%]                        46.511628
Best Trade [%]                      17.704985
Worst Trade [%]                    -30.451034
Avg. Trade [%]                    

In [136]:
backtest.plot()

Row(id='14240', ...)

## MA Strategy

In [132]:
backtest_MA = Backtest(df, MySMAStrategy, cash=10_000, commission = 0.002, exclusive_orders=True)
stats_MA = backtest_MA.run()
stats_MA

Start                     2013-12-16 00:00:00
End                       2023-01-31 00:00:00
Duration                   3333 days 00:00:00
Exposure Time [%]                   98.781019
Equity Final [$]                 18901.458446
Equity Peak [$]                  26943.693489
Return [%]                          89.014584
Buy & Hold Return [%]              624.685208
Return (Ann.) [%]                    7.234328
Volatility (Ann.) [%]               30.205252
Sharpe Ratio                         0.239506
Sortino Ratio                        0.384215
Calmar Ratio                         0.179781
Max. Drawdown [%]                  -40.239695
Avg. Drawdown [%]                   -5.891325
Max. Drawdown Duration     1879 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                  107
Win Rate [%]                        37.383178
Best Trade [%]                      79.788108
Worst Trade [%]                    -13.977958
Avg. Trade [%]                    

In [133]:
backtest_MA.plot()

Row(id='13483', ...)